In [17]:
%reset-f
%load_ext autoreload
%autoreload
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit
from tools import *

os.makedirs("fig",exist_ok=True)

### Piecewise linear trajectory

In [19]:
def traj(x,s,t,u,a,m):
    #print("in",(x,s,t,u,a,m))
    u=2030 #forces u to be last value, this is equivalent siince we still chose a
    return extrap(x,[0,s,t,u],[0,0,a,max(m,a)]) #forces positive slope

### Data

In [24]:
#Reads abatement the data
df_in=pd.read_csv("input/abat.csv").set_index("year")

#Rescales the data
power = (np.floor(np.log10(df_in.max().max())/3))*3
print("scaling at x10^-"+str(power))
abat_xl=df_in * 10**(-power);

year=df_in.index.values
print(year)

#Cost data
macs = macs = pd.read_csv("input/mac_names.csv").set_index("measure")
    

scaling at x10^-6.0
[2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019
 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030]


In [30]:
#Output dataframe
df = pd.DataFrame(index=df_in.columns,columns=["speed","MAC","potential","t","growth"])
df["MAC"]=macs

### Loop on measures

In [54]:
for j in df.index:
#for j in ["Industrial wastewater"]:
    y=abat_xl.ix[:,j].interpolate() #fills missing values by interpolating

    #guess
    s_ = year[3];
    t_ = year[15];
    u_=year[-1];
    a_ = y[t_];
    m_ = y[year[-1]]
    
    #fit
    (s,t,u,a,m),foo =curve_fit(traj, year, y, maxfev=int(1e4), p0 = [s_, t_,u_,a_,m_])
   
    #makes sure start is before t
    s,t = min(s,t),max(s,t)
    
    #plot
    plt.scatter(year,y,c="red")
    plt.plot(year,traj(year,s,t,u,a,m))
    
    #speeds
    if abs(s-t)<2: #case with a jump
        va= a
    else:
        va = a/(t-s)
        
    vb = (m-a)/(u-t)
    
    if va>vb: #classic case
        df["t"]=t
        df["speed"]=va
        df["growth"]=vb
        df["potential"]=a
    else:
        v=m/(u-s)
        df["t"]=s
        df["speed"]=v
        df["growth"]=v
        df["potential"]=0
        
        plt.plot(year,traj(year,s,s,u,0,v*(u_-s)),color="green")

    #saves figures and clears
    plt.savefig("fig/"+j+".png")
    plt.close()
    
        
        
    
   
    
    

In [51]:
(s,t,u,a,m,va,vb)

(2009.0126185362617,
 2009.5895793414882,
 2030.0,
 7.6873996154793067,
 14.683664853889249,
 13.323954670477008,
 0.3427790811108179)

In [48]:
min(s,t)

2009.0126185362617

In [50]:
s

2009.0126185362617